# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
B. Maity  ->  B. Maity  |  ['B. Maity']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 77 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.11612


extracting tarball to tmp_2507.11612... done.


Found 77 bibliographic references in tmp_2507.11612/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2507.11618


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Keppler_2018}\bibinfo{author}{{Keppler}, M.} \emph{et~al.}\newblock \bibinfo{title}{{Discovery of a planetary-mass companion within the  gap of the transition disk around PDS 70}}.\newblock \emph{\bibinfo{journal}{\aap}} \textbf{\bibinfo{volume}{617}},  \bibinfo{pages}{A44} (\bibinfo{year}{2018}).
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Haffert_2019}\bibinfo{author}{{Haffert}, S.~Y.} \emph{et~al.}\newblock \bibinfo{title}{{Two accreting protoplanets around the young star PDS  70}}.\newblock \emph{\bibinfo{journal}{\nastro}} \textbf{\bibinfo{volume}{3}},  

...

✔ → 88618:\section{Retrieval results}
  ↳ 96010:\section{Discussion}


✔ → 96010:\section{Discussion}
  ↳ 110892:\section{Conclusion}
✔ → 110892:\section{Conclusion}
  ↳ 115570:\begin{appendix}
✔ → 115570:\begin{appendix}
  ↳ 115587:\section{Technical details on NPE}
✔ → 115587:\section{Technical details on NPE}
  ↳ 120705:\section{Diagnostics}


✔ → 120705:\section{Diagnostics}
  ↳ 135438:\begin{appendix}
✔ → 135438:\begin{appendix}
  ↳ 135455:\section{Technical details on NPE}
✔ → 135455:\section{Technical details on NPE}
  ↳ 140573:\section{Diagnostics}


✔ → 140573:\section{Diagnostics}
  ↳ 155305:end


Found 73 bibliographic references in tmp_2507.12264/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2507.12315
extracting tarball to tmp_2507.12315... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.11612-b31b1b.svg)](https://arxiv.org/abs/2507.11612) | **A young gas giant and hidden substructures in a protoplanetary disk**  |
|| Á. Ribas, et al. -- incl., <mark>F. Zagaria</mark> |
|*Appeared on*| *2025-07-17*|
|*Comments*| *Nature Astronomy, 13 pages and 4 figures + Supplementary material*|
|**Abstract**|            The detection of planets in protoplanetary disks has proven to be extremely challenging. In contrast, rings and gaps, usually attributed to planet-disk interactions, have been found in virtually every large protoplanetary (Class II) disk observed at 0.9-1.3 mm with sufficient spatial resolution (5 au). The nearby disk around MP Mus (PDS 66) stands as an exception to this rule, and its advanced age (7-10 Myr) is particularly difficult to reconcile with its apparent lack of substructures. Despite the disk's smooth appearance, Gaia data of MP Mus show a significant proper motion anomaly, signalling the presence of a companion. Here we present ALMA 3 mm observations of the system with comparable high spatial resolution to previous 1.3 mm data. The new observations pierce deeper into the disk midplane and reveal an inner cavity (<3 au) and a ring at 10 au. The disk structure inferred from ALMA observations narrows down the properties of the companion to a gas giant orbiting at 1-3 au, and hydrodynamic simulations further confirm that such a planet can produce the observed cavity. These independent pieces of evidence constitute an indirect but compelling detection of an exoplanet within a protoplanetary disk using Gaia astrometry. MP Mus is the first system in which undetected substructures are revealed thanks to the lower optical depths at longer wavelengths, suggesting that rings and gaps are even more abundant than what is currently believed.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.11669-b31b1b.svg)](https://arxiv.org/abs/2507.11669) | **exoALMA. XVIII. Interpreting large scale kinematic structures as moderate warping**  |
|| A. J. Winter, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark> |
|*Appeared on*| *2025-07-17*|
|*Comments*| *Accepted for publication in ApJL as part of the exoALMA series. 53 pages, 37 figures*|
|**Abstract**|            The exoALMA program gave an unprecedented view of the complex kinematics of protoplanetary disks, revealing diverse structures that remain poorly understood. We show that moderate disk warps ($\sim 0.5-2^\circ$) can naturally explain many of the observed large-scale velocity features with azimuthal wavenumber $m = 1$. Using a simple model, we interpret line-of-sight velocity variations as changes in the projected Keplerian rotation caused by warping of the disk. While not a unique explanation, this interpretation aligns with growing observational evidence that warps are common. We demonstrate that such warps can also produce spiral structures in scattered light and CO brightness temperature, with $\sim 10$ K variations in MWC 758. Within the exoALMA sample, warp properties correlate with stellar accretion rates, suggesting a link between the inner disc and outer disc kinematics. If warps cause large-scale kinematic structure, this has far reaching implications for turbulence, angular momentum transport, and planet formation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12116-b31b1b.svg)](https://arxiv.org/abs/2507.12116) | **Euclid preparation. Simulating thousands of Euclid spectroscopic skies**  |
|| E. Collaboration, et al. -- incl., <mark>Y. Wang</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-07-17*|
|*Comments*| *19 pages, submitted to A&A*|
|**Abstract**|            We present two extensive sets of 3500+1000 simulations of dark matter haloes on the past light cone, and two corresponding sets of simulated (`mock') galaxy catalogues that represent the Euclid spectroscopic sample. The simulations were produced with the latest version of the PINOCCHIO code, and provide the largest, public set of simulated skies. Mock galaxy catalogues were obtained by populating haloes with galaxies using an halo occupation distribution (HOD) model extracted from the Flagship galaxy catalogue provided by Euclid Collaboration. The Geppetto set of 3500 simulated skies was obtained by tiling a 1.2 Gpc/h box to cover a light-cone whose sky footprint is a circle of 30 deg radius, for an area of 2763 deg$^2$ and a minimum halo mass of $1.5\times10^{11}$ Msun/h. The relatively small box size makes this set unfit for measuring very large scales. The EuclidLargeBox set consists of 1000 simulations of 3.38 Gpc/h, with the same mass resolution and a footprint that covers half of the sky, excluding the Milky Way zone of avoidance. From this we produced a set of 1000 EuclidLargeMocks on the 30 deg radius footprint, whose comoving volume is fully contained in the simulation box. We validated the two sets of catalogues by analysing number densities, power spectra, and 2-point correlation functions, showing that the Flagship spectroscopic catalogue is consistent with being one of the realisations of the simulated sets, although we noticed small deviations limited to the quadrupole at k>0.2 h/Mpc. We show cosmological parameter inference from these catalogues and demonstrate that using one realisation of EuclidLargeMocks in place of the Flagship mock produces the same posteriors, to within the expected shift given by sample variance. These simulated skies will be used for the galaxy clustering analysis of Euclid's Data Release 1 (DR1).         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12170-b31b1b.svg)](https://arxiv.org/abs/2507.12170) | **Measuring the IGM correlation length at 5<z<6.1: a fast change at the end of Reionization**  |
|| B. Spina, S. E. I. Bosman, <mark>B. Maity</mark>, F. B. Davies |
|*Appeared on*| *2025-07-17*|
|*Comments*| *11 pages, 5 figures, 1 table. Submitted to A&A*|
|**Abstract**|            The Lyman-$\alpha$ forest of high redshift quasars is a powerful probe of the late stages of the Epoch of Reionization (EoR), particularly through the presence of Gunn-Peterson troughs. These troughs span a broad range of lengths (up to $\sim 100$ Mpc), suggesting large-scale coherent structures in the intergalactic medium. We aim to gain insight into the presence, extent, and magnitude of correlations in the Lyman-$\alpha$ forest at $5<z<6.1$, and to quantify the scales over which correlations are significant to inform on the necessary volume for simulating the EoR. Using the extended XQR-30 dataset and accounting for the relevant systematics, we compute the flux correlation matrix and perform an MCMC analysis to quantify the extent and strength of the correlation. To interpret our results, we compare to $1.5^3,\mathrm{Gpc}^3$ lightcones of Lyman-$\alpha$ transmission fluxes from several reionization scenarios, including a fiducial box employing SCRIPT. We detect strong correlations at $z > 5.3$, extending at least tens of Mpc and strongly increasing with redshift. Our results suggest a redshift-dependent correlation length, from $L \leq 26.53\, (68.47)$ Mpc at 1-$\sigma$ (2-$\sigma$) limit at $z = 5.0$ to $L = 252.72^{+272.61}_{-41.61}$ Mpc at $z = 6.1$. In contrast, all simulation models predict shorter correlation lengths ($< 60$ Mpc) and a slower redshift evolution. The presence and redshift-dependence of correlations in the Lyman-$\alpha$ forest on $>200$ Mpc scales at $z=6$ indicates that cosmological simulations should be larger than this scale to adequately sample the Lyman-$\alpha$ forest. Despite implementing a fluctuating UVB and numerous neutral islands at $z<6$, our fiducial SCRIPT-based simulation fails to reproduce the large-scale correlations. It may be that those ingredients are necessary, but not sufficient, to understanding the unfolding of the EoR.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12264-b31b1b.svg)](https://arxiv.org/abs/2507.12264) | **Panchromatic characterization of the Y0 brown dwarf WISEP J173835.52+273258.9 using JWST/MIRI**  |
|| M. Vasist, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-07-17*|
|*Comments*| **|
|**Abstract**|            Cold brown dwarf atmospheres are good training grounds for analyzing temperate giant planets. WISEP J173835.52+273258.9 (WISE 1738) is an isolated Y0 brown dwarf with a temperature between 350-400 K, at the T-Y transition. While its near-infrared spectrum has been studied, bulk properties and chemistry remain uncertain. We analyze new JWST MIRI medium-resolution spectra (5-18 micron), combined with near-infrared spectra (0.98-2.2 micron) from HST/WFC3 and Gemini/GNIRS, to better constrain WISE 1738's atmosphere and physical parameters. We use Neural Posterior Estimation (NPE) with a cloud-free petitRADTRANS model and evaluate results using posterior checks, coverage, and L-C2ST diagnostics. Our retrieval confirms previous constraints on H2O, CH4, and NH3, and for the first time constrains CO, CO2, and 15NH3. We find evidence of disequilibrium chemistry through CO and CO2 abundances not expected under equilibrium. Estimated properties are temperature 402 (+12,-9) K, log g 4.43 (+0.26,-0.34) cm/s2, mass 13 (+11,-7) M_Jup, radius 1.14 (+0.03,-0.03) R_Jup, and bolometric luminosity -6.52 (+0.05,-0.04) log L/L_sun. Evolutionary models suggest an age between 1 and 4 Gyr, consistent with a 6-hour rotation. We place an upper bound on 15NH3, implying a 3-sigma lower limit on the 14N/15N ratio of 275. We also derive a C/O ratio of 1.35 (+0.39,-0.31) and metallicity of 0.34 (+0.12,-0.11), without accounting for oxygen sequestration.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.11618-b31b1b.svg)](https://arxiv.org/abs/2507.11618) | **Double White Dwarf Binaries in SDSS-V DR19 : The discovery of a rare DA+DQ white dwarf binary with 31 hour orbital period**  |
|| G. A. Pallathadka, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-07-17*|
|*Comments*| *Submitted to AAS journals*|
|**Abstract**|            Binaries of two white dwarfs (WDs) are an important class of astrophysical objects that are theorized to lead to Type Ia supernovae and are also used to gain insight into complex processes involved in stellar binary evolution. We report the discovery of SDSS~J090618.44+022311.6, a rare post-common envelope binary of a hydrogen atmospheric DA WD and a DQ WD which shows carbon absorption features, and is only the fourth such binary known. We combine the available spectroscopic, photometric, and radial velocity data to provide a self-consistent model for the binary and discuss its history as a binary DA+DQ. The system has a period of 31.17 hours with masses of 0.42 M$_{\odot}$ for DA WD and 0.49 M$_{\odot}$ for DQ WD. The corresponding cooling ages point to an Algol type of evolution with the lower mass star evolving into a DA WD first and later the massive DQ WD is formed. The system has a merger timescale of 450 Gyrs and will lead to the formation of a massive WD. With this, the number of known DA+DQ WD binaries has increased to four, and we find that their stellar properties all lie in the same range. Detailed study of more such systems is vital to understand common processes involved in the formation of this rare class of binaries and give insights towards the broader picture of WD spectral evolution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.12315-b31b1b.svg)](https://arxiv.org/abs/2507.12315) | **First Constraints from Marked Angular Power Spectra with Subaru Hyper Suprime-Cam Survey First-Year Data**  |
|| J. A. Cowell, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-07-17*|
|*Comments*| *Submitted to MNRAS. Comments welcome*|
|**Abstract**|            We present the first application of marked angular power spectra to weak lensing data, using maps from the Subaru Hyper Suprime-Cam Year 1 (HSC-Y1) survey. Marked convergence fields, constructed by weighting the convergence field with non-linear functions of its smoothed version, are designed to encode higher-order information while remaining computationally tractable. Using simulations tailored to the HSC-Y1 data, we test three mark functions that up- or down-weight different density environments. Our results show that combining multiple types of marked auto- and cross-spectra improves constraints on the clustering amplitude parameter $S_8\equiv\sigma_8\sqrt{\Omega_{\rm m}/0.3}$ by $\approx$43\% compared to standard two-point power spectra. When applied to the HSC-Y1 data, this translates into a constraint on $S_8 = 0.807\pm 0.024$. We assess the sensitivity of the marked power spectra to systematics, including baryonic effects, intrinsic alignment, photometric redshifts, and multiplicative shear bias. These results demonstrate the promise of marked statistics as a practical and powerful tool for extracting non-Gaussian information from weak lensing surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.11612/./figures/PMa_combined_fig1.png', 'tmp_2507.11612/./figures/B3_and_B6_fig2.png', 'tmp_2507.11612/./figures/rad_profiles_frank_fig3.png']
copying  tmp_2507.11612/./figures/PMa_combined_fig1.png to _build/html/
copying  tmp_2507.11612/./figures/B3_and_B6_fig2.png to _build/html/
copying  tmp_2507.11612/./figures/rad_profiles_frank_fig3.png to _build/html/
exported in  _build/html/2507.11612.md
    + _build/html/tmp_2507.11612/./figures/PMa_combined_fig1.png
    + _build/html/tmp_2507.11612/./figures/B3_and_B6_fig2.png
    + _build/html/tmp_2507.11612/./figures/rad_profiles_frank_fig3.png
found figures ['tmp_2507.11669/./combined_model_plot_MWC758.png', 'tmp_2507.11669/./correlations_figure_beta.png', 'tmp_2507.11669/./combined_model_plot_AATau.png']
copying  tmp_2507.11669/./combined_model_plot_MWC758.png to _build/html/
copying  tmp_2507.11669/./correlations_figure_beta.png to _build/html/
copying  tmp_2507.11669/./combined_model_plot_AATau.png to _build/h

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\ac}{Astron. Comput.}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nat}{Nature}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\pasa}{Publ. Astron. Soc. Aust.}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\sci}{Science}$</div>



<div id="title">

# A young gas giant and hidden substructures in a protoplanetary disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.11612-b31b1b.svg)](https://arxiv.org/abs/2507.11612)<mark>Appeared on: 2025-07-17</mark> -  _Nature Astronomy, 13 pages and 4 figures + Supplementary material_

</div>
<div id="authors">

Á. Ribas, et al. -- incl., <mark>F. Zagaria</mark>

</div>
<div id="abstract">

**Abstract:** The detection of planets in protoplanetary disks has proven to be extremely challenging. In contrast, rings and gaps, usually attributed to planet-disk interactions, have been found in virtually every large protoplanetary (Class II) disk observed at 0.9-1.3 mm with sufficient spatial resolution (5 au). The nearby disk around MP Mus (PDS 66) stands as an exception to this rule, and its advanced age (7-10 Myr) is particularly difficult to reconcile with its apparent lack of substructures. Despite the disk's smooth appearance, _Gaia_ data of MP Mus show a significant proper motion anomaly, signalling the presence of a companion. Here we present ALMA 3 mm observations of the system with comparable high spatial resolution to previous 1.3 mm data. The new observations pierce deeper into the disk midplane and reveal an inner cavity ( $<$ 3 au) and a ring at 10 au. The disk structure inferred from ALMA observations narrows down the properties of the companion to a gas giant orbiting at 1-3 au, and hydrodynamic simulations further confirm that such a planet can produce the observed cavity. These independent pieces of evidence constitute an indirect but compelling detection of an exoplanet within a protoplanetary disk using _Gaia_ astrometry. MP Mus is the first system in which undetected substructures are revealed thanks to the lower optical depths at longer wavelengths, suggesting that rings and gaps are even more abundant than what is currently believed.

</div>

<div id="div_fig1">

<img src="tmp_2507.11612/./figures/PMa_combined_fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Orbital separation and mass of a companion that could produce the _Gaia_ proper motion anomaly of MP Mus. The left panel shows the individual _Gaia_ simulations which produce an astrometric signal compatible with the observations ($0.26>|\Delta \mu|>0.16$ mas yr$ ^{-1}$, $|\Delta \mu|/\sigma_{|\Delta \mu|}>4$, and $\rm{UWE_{DR3}}<1.1$). The right panel presents the same simulations but smoothed over a 2d histogram (contours correspond to 0.05, 0.15, and 0.2 \%). When considering the disk structure traced in the millimetre (shaded area), the most likely companion is a gas giant at 1-2 au. Crosses indicate the parameters used for the hydrodynamical simulations in Sec. \ref{sec:simulations}. The simulations that best reproduce both _Gaia_ and ALMA observations are show in red (Sec. \ref{sec:discussion}). (*fig:PMa*)

</div>
<div id="div_fig2">

<img src="tmp_2507.11612/./figures/B3_and_B6_fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** ALMA continuum observations of MP Mus at 3 mm and 1.3 mm. The solid and dashed white arcs in the 3 mm image mark the ring and two gaps. The beam of each observation is indicated as a white ellipse in the bottom left corners. (*fig:ALMA*)

</div>
<div id="div_fig3">

<img src="tmp_2507.11612/./figures/rad_profiles_frank_fig3.png" alt="Fig3" width="100%"/>

**Figure 3. -** Intensity radial profiles of MP Mus derived using frank. The 3 mm profile shows structures undetected at shorter wavelengths, including an inner cavity, a ring, and two gaps. These features are indicated in the figure. (*fig:rad_profiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.11612"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cp}[1]{\textcolor{red}{{CP:} #1}}$
$\newcommand{\InstMPIA}{Max-Planck Institute for Astronomy (MPIA), Königstuhl 17, 69117 Heidelberg, Germany}$
$\newcommand{\InstMIT}{Department of Earth, Atmospheric, and Planetary Sciences, Massachusetts Institute of Technology, Cambridge, MA 02139, USA}$
$\newcommand{\InstOCA}{Université Côte d’Azur, Observatoire de la Côte d’Azur, CNRS, Laboratoire Lagrange, France}$
$\newcommand{\InstMilano}{Dipartimento di Fisica, Università degli Studi di Milano, Via Celoria 16, 20133 Milano, Italy}$
$\newcommand{\InstNAOJ}{National Astronomical Observatory of Japan, Osawa 2-21-1, Mitaka, Tokyo 181-8588, Japan}$
$\newcommand{\InstIPAGGrenoble}{Univ. Grenoble Alpes, CNRS, IPAG, 38000 Grenoble, France}$
$\newcommand{\InstMonash}{School of Physics and Astronomy, Monash University, Clayton VIC 3800, Australia}$
$\newcommand{\InstCfA}{Center for Astrophysics | Harvard \& Smithsonian, Cambridge, MA 02138, USA}$
$\newcommand{\InstFlorida}{Department of Astronomy, University of Florida, Gainesville, FL 32611, USA}$
$\newcommand{\InstChile}{Departamento de Astronomía, Universidad de Chile, Camino El Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\InstStAndrewsPhysics}{School of Physics \& Astronomy, University of St. Andrews, North Haugh, St. Andrews KY16 9SS, UK}$
$\newcommand{\InstStAndrewsExoplanets}{Centre for Exoplanet Science, University of St. Andrews, North Haugh, St. Andrews, KY16 9SS, UK}$
$\newcommand{\InstRicePhysics}{Department of Physics and Astronomy, Rice University, Houston, TX 77005, USA}$
$\newcommand{\InstLANL}{Los Alamos National Laboratory, Los Alamos, NM 87545, USA}$
$\newcommand{\InstUGAphysics}{Department of Physics and Astronomy, The University of Georgia, Athens, GA 30602, USA}$
$\newcommand{\InstUGACSP}{Center for Simulational Physics, The University of Georgia, Athens, GA 30602, USA}$
$\newcommand{\InstUGAIA}{Institute for Artificial Intelligence, The University of Georgia, Athens, GA, 30602, USA}$
$\newcommand{\InstColumbia}{Department of Astronomy, Columbia University, 538 W. 120th Street, Pupin Hall, New York, NY, USA}$
$\newcommand{\InstLeeds}{School of Physics and Astronomy, University of Leeds, Leeds, UK, LS2 9JT}$
$\newcommand{\InstRiceSpace}{Rice Space Institute, Rice University, 6100 Main St, Houston, TX 77005, USA}$
$\newcommand{\InstLeiden}{Leiden Observatory, Leiden University, P.O. Box 9513, NL-2300 RA Leiden, The Netherlands}$
$\newcommand{\InstESO}{European Southern Observatory, Karl-Schwarzschild-Str. 2, D-85748 Garching bei München, Germany}$
$\newcommand{\InstNHFP}{NASA Hubble Fellowship Program Sagan Fellow}$
$\newcommand{\InstIbaraki}{College of Science, Ibaraki University, 2-1-1 Bunkyo, Mito, Ibaraki 310-8512, Japan}$
$\newcommand{\InstCambridge}{Institute of Astronomy, University of Cambridge, Madingley Road, CB3 0HA, Cambridge, UK}$
$\newcommand{\InstNRAO}{National Radio Astronomy Observatory, 520 Edgemont Rd., Charlottesville, VA 22903, USA}$
$\newcommand{\InstUNAM}{Instituto de Ciencias Físicas, Universidad Nacional Autónoma de México, Av. Universidad s/n, 62210 Cuernavaca, Mor., Mexico}$
$\newcommand{\InstBologna}{Alma Mater Studiorum Università di Bologna, Dipartimento di Fisica e Astronomia (DIFA), Via Gobetti 93/2, 40129 Bologna, Italy}$
$\newcommand{\InstArcetri}{INAF-Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, 50125 Firenze, Italy}$
$\newcommand{\InstASIAA}{Academia Sinica Institute of Astronomy \& Astrophysics, 11F of Astronomy-Mathematics Building, AS/NTU, No.1, Sec. 4, Roosevelt Rd, Taipei 10617, Taiwan}$
$\newcommand{\InstWesleyan}{Department of Astronomy, Van Vleck Observatory, Wesleyan University, 96 Foss Hill Drive, Middletown, CT 06459, USA}$
$\newcommand{\InstPennState}{Department of Astronomy \& Astrophysics, 525 Davey Laboratory, The Pennsylvania State University, University Park, PA 16802, USA}$
$\newcommand{\InstSOKENDAI}{Department of Astronomical Science, The Graduate University for Advanced Studies, SOKENDAI, 2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}$
$\newcommand{\InstQMUL}{Astronomy Unit, School of Physics and Astronomy, Queen Mary University of London, London E1 4NS, UK}$
$\newcommand{\thefigure}{A\arabic{figure}}$
$\newcommand{\thefigure}{B\arabic{figure}}$
$\newcommand{\thefigure}{C\arabic{figure}}$</div>



<div id="title">

# exoALMA. XVIII. Interpreting large scale kinematic structures as moderate warping

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.11669-b31b1b.svg)](https://arxiv.org/abs/2507.11669)<mark>Appeared on: 2025-07-17</mark> -  _Accepted for publication in ApJL as part of the exoALMA series. 53 pages, 37 figures_

</div>
<div id="authors">

A. J. Winter, et al. -- incl., <mark>M. Benisty</mark>, <mark>D. Fasano</mark>

</div>
<div id="abstract">

**Abstract:** The exoALMA program gave an unprecedented view of the complex kinematics of protoplanetary disks, revealing diverse structures that remain poorly understood. We show that moderate disk warps ( $\sim 0.5-2^\circ$ ) can naturally explain many of the observed large-scale velocity features with azimuthal wavenumber $m = 1$ . Using a simple model, we interpret line-of-sight velocity variations as changes in the projected Keplerian rotation caused by warping of the disk. While not a unique explanation, this interpretation aligns with growing observational evidence that warps are common. We demonstrate that such warps can also produce spiral structures in scattered light and CO brightness temperature, with $\sim 10$ K variations in MWC 758. Within the exoALMA sample, warp properties correlate with stellar accretion rates, suggesting a link between the inner disc and outer disc kinematics. If warps cause large-scale kinematic structure, this has far reaching implications for turbulence, angular momentum transport, and planet formation.

</div>

<div id="div_fig1">

<img src="tmp_2507.11669/./combined_model_plot_MWC758.png" alt="Fig36" width="100%"/>

**Figure 36. -** Top panels show residuals from the observed (left) vs modeled (right) $\delta v_{\rm los}$ fields for MWC 758 after fitting Keplerian velocity profiles. The flexible model is for a simple warped disc geometry, with perturbation in inclination and position angle. The colour scale is the LOS velocity  in km/s. Grey circles mask two times the central beam size. The beam size is also shown on the left hand side, assumed circular for visualisation. The bottom panels show radial profiles of \( \delta i \), \( \delta \mathrm{PA} \) and the physical warp properties tilt $\beta$, twist $\gamma$ and the warp amplitude $\psi$ for MWC 758 from our fitting procedure. Blue points and errorbars in \( \delta i \) and \( \delta \mathrm{PA} \) come from the least squares fitting procedure, Faint orange lines show posterior distributions from the GP model. (*fig:inc_pa_profiles*)

</div>
<div id="div_fig2">

<img src="tmp_2507.11669/./correlations_figure_beta.png" alt="Fig4" width="100%"/>

**Figure 4. -** From top to bottom, we show how the the continuum non-axisymmetric index \citep[NAI, top --][]{Curone_exoALMA}, stellar accretion rates (middle top), normalised stellar accretion rates (to the square of the stellar mass, bottom middle) and NIR excess \citep[][bottom]{Garufi_ea_2018} depend on the range of tilt amplitudes we infer from our model.
    The outcome of Spearman rank, Kendall $\tau$ and permutation correlation tests are shown in terms of correlation statistic and $p$-value.
    We exclude cases where inclination is unfavourable due to the appearance of the back-side of the disc. (*fig:correlations*)

</div>
<div id="div_fig3">

<img src="tmp_2507.11669/./combined_model_plot_AATau.png" alt="Fig8" width="100%"/>

**Figure 8. -** As in Figure \ref{fig:inc_pa_profiles} but for AA Tau. (*fig:profile_AATau*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.11669"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pigi}[1]{\textcolor{orange}{ [Pigi: #1]}}$
$\newcommand{\fha}{\ensuremath{f_{{\rm H}\alpha}}\xspace}$
$\newcommand{\vx}{\vec{x}\xspace}$
$\newcommand{\nh}{\hat{n}\xspace}$
$\newcommand{\hmpc}{\ensuremath{h^{-1} \text{Mpc}}\xspace}$
$\newcommand{\kmpc}{\ensuremath{h \text{Mpc}^{-1}}\xspace}$
$\newcommand{\dens}{\ensuremath{h^{3} \mathrm{Mpc}^{-3}}\xspace}$
$\newcommand{\cgpc}{\ensuremath{h^{-3} \mathrm{Gpc}^3}\xspace}$
$\newcommand{\hgpc}{\ensuremath{h^{-1} \text{Gpc}}\xspace}$
$\newcommand{\hmsun}{\ensuremath{h^{-1} M_\odot}\xspace}$
$\newcommand{\Comp}{{\cal{C}}}$
$\newcommand{\Compa}{\tilde{\cal C}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}\xspace}$
$\newcommand{\pin}{\texttt{Pinocchio}\xspace}$
$\newcommand{\tdeg}{\ensuremath{30^{\circ}}\xspace}$
$\newcommand{\sqdeg}{\ensuremath{{\rm deg}^2}\xspace}$
$\newcommand{\gl}[1]{\textcolor{red}{[{\bf Guilhem}: #1]}}$
$\newcommand{\glrm}[1]{\textcolor{red}{[{\bf Guilhem}: \sout{#1}]}}$
$\newcommand{\tc}[1]{\textcolor{blue}{[{\bf Tiago}: #1]}}$
$\newcommand{\MYK}[1]{\textcolor{magenta}{[{\bf Yousry}: #1]}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation: Simulating thousands of $\Euclid$ spectroscopic skies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.12116-b31b1b.svg)](https://arxiv.org/abs/2507.12116)<mark>Appeared on: 2025-07-17</mark> -  _19 pages, submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>Y. Wang</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We present two extensive sets of 3500 $+$ 1000 simulations of dark matter haloes  on the past light cone, and two corresponding sets of simulated (`mock') galaxy  catalogues that represent the $\Euclid$ spectroscopic sample. The simulations were  produced with the latest version of the ${\pin}$ code, and provide the largest, public set  of simulated skies. Mock galaxy catalogues were obtained by populating haloes with  galaxies using an halo occupation distribution (HOD) model extracted from the Flagship  galaxy catalogue provided by Euclid Collaboration. The Geppetto set of 3500 simulated  skies was obtained by tiling a $1.2 \hgpc$ box to cover a light-cone whose sky  footprint is a circle of ${\tdeg}$ radius, for an area of 2763 $\sqdeg$ and a minimum halo  mass of $1.5\times10^{11}$ $\hmsun$ . The relatively small box size makes this set unfit  for measuring very large scales. The EuclidLargeBox set consists of 1000 simulations of $3.38 \hgpc$ , with the same mass resolution and a footprint that covers half of the  sky, excluding the Milky Way zone of avoidance. From this we produced a set of 1000  EuclidLargeMocks on the ${\tdeg}$ radius footprint, whose comoving volume is fully  contained in the simulation box. We validated the two sets of catalogues by analysing  number densities, power spectra, and 2-point correlation functions, showing that the  Flagship spectroscopic catalogue is consistent with being one of the realisations of the  simulated sets, although we noticed small deviations limited to the quadrupole at $k>0.2 \kmpc$ . We show cosmological parameter inference from these catalogues and  demonstrate that using one realisation of EuclidLargeMocks in place of the Flagship mock  produces the same posteriors, to within the expected shift given by sample variance.  These simulated skies will be used for the galaxy clustering analysis of $\Euclid$ 's Data  Release 1 (DR1).

</div>

<div id="div_fig1">

<img src="tmp_2507.12116/./Figures/30degCone_replicas.png" alt="Fig1" width="100%"/>

**Figure 1. -** Graphical representation of the comoving volume of the {$\tdeg$} radius survey
  footprint, highlighting the four redshift bins bounded by redshifts
  $[0.9,1.1,1.3,1.5,1.8]$, immersed in the comoving volume of the Flagship simulation
  (that is our reference), of EuclidLargeBox, and of Geppetto simulation boxes. The
  Geppetto box is tiled a few times to cover the survey volume; for sake of clarity, the
  tiling is limited to the main replications. Here the central axis of the footprint lies
  on the main diagonal of the box, however all EuclidLargeBox and most Geppetto boxes have
  random orientations of this axis. (*fig:replicas*)

</div>
<div id="div_fig2">

<img src="tmp_2507.12116/./Figures/haloMF.png" alt="Fig2" width="100%"/>

**Figure 2. -** Halo mass function $n(M_{\rm h})$ of the simulated sets. The upper panel only
  shows the mass function of EuclidLargeBox (ELB) for $z=0$ and $z=1$, where the line
  gives the average over the 1000 simulations and the shaded area gives its sample
  variance. The two lower panels show relative residuals, $n/n_{\rm elb}-1$ in percent, of
  the mass function of the first 600 Geppetto simulations, run with V4 of the code (Gep
  V4), and the other 2900, run with V5 (Gep V5), with respect to the EuclidLargeBox
  measurement, respectively at $z=0$(mid panel) and $z=1$(lower panel). Here the
  reported sample variance is relative only to the Geppetto sets. (*fig:haloMF*)

</div>
<div id="div_fig3">

<img src="tmp_2507.12116/./Figures/PKreplicas.png" alt="Fig3" width="100%"/>

**Figure 3. -** Power spectrum monopole of {$\pin$} dark matter haloes from the Geppetto
  light-cones, with $M_{\rm h}>10^{12}$\hmsun in the redshift range $z\in[0.9,1.1]$. The
  continuous lines show measurements in a box of $L_{\rm m}=7.2 $\hgpc$$ sampled on its
  fundamental frequency $k_{\rm fm}$, the dashed lines show the same measurements with a
  sampling of $6k_{\rm fm}$, equal to the fundamental mode of the simulation box $k_{\rm
    fs}$. Blue and orange lines show respectively averages over 100 mocks with random
  orientations of the cone axis and with axis aligned with the main diagonal of the
  simulation box. The lower panel shows the residuals of the measurements along a random
  orientation with respect to those with axis along the main diagonal, plus, in red, the
  residuals of the two rebinned measurements. (*fig:PKreplicas*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.12116"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcidauthorA}{0000-0003-1634-1283}$
$\newcommand{\orcidauthorB}{0000-0001-8582-7012}$
$\newcommand{\orcidauthorC}{0000-0003-0821-3644}$
$\newcommand{\orcidauthorD}{0000-0002-4682-6970}$
$\newcommand{\bene}{\textcolor{blue}}$
$\newcommand{\BM}[1]{{\color{magenta}[{BM: }#1]}}$
$\newcommand{\arraystretch}{1.8}$</div>



<div id="title">

# Measuring the IGM correlation length at 5<z<6.1: a fast change at the end of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.12170-b31b1b.svg)](https://arxiv.org/abs/2507.12170)<mark>Appeared on: 2025-07-17</mark> -  _11 pages, 5 figures, 1 table. Submitted to A&A_

</div>
<div id="authors">

B. Spina, S. E. I. Bosman, <mark>B. Maity</mark>, F. B. Davies

</div>
<div id="abstract">

**Abstract:** The Lyman- $\alpha$ forest of high redshift quasars is a powerful probe of the late stages of the Epoch of Reionization (EoR), in particular thanks to the presence of Gunn-Peterson troughs. These troughs exhibit a broad range of lengths, with some extending up to $\sim 100$ cMpc, suggesting large-scale coherent structures in the intergalactic medium (IGM). We aim to gain more insight into the presence, extent, and magnitude of correlations in the Lyman- $\alpha$ forest during the end of reionization, at $5<z<6.1$ . In particular, we want to quantify the scales over which correlations are significant in order to help inform the required size of cosmological simulations aiming to capture the evolution of the EoR. We utilize the extended XQR-30 dataset over the redshift range $5 < z < 6.1$ ( $\Delta z = 0.05$ ) to explore large-scale correlations. After accounting for the relevant systematics, the flux correlation matrix proves to be a powerful tool for probing large-scale correlations across redshifts. We perform an MCMC analysis to quantify the extent and strength of the correlation, making use of several functional forms. We moreover employ new large-volume ( $1.5^3 \mathrm{Gpc}^3$ ) light-cones of Lyman- $\alpha$ transmission implementing different reionization scenarios to interpret the observed signal, including a fiducial box employing SCRIPT. We detect strong correlations at redshifts $z > 5.3$ , extending at least tens of Mpc and strongly increasing with redshift. Our results suggest a redshift-dependent correlation length, from $L \leq 26.53  (68.47) \mathrm{Mpc}$ at 1- $\sigma$ (2- $\sigma$ ) limit at redshift $z = 5.0$ to $L = 252.72^{+272.61}_{-41.61} \mathrm{Mpc}$ at redshift $z = 6.1$ . On the contrary, our simulations all demonstrate characteristic correlation scales $< 60$ Mpc with a very slow redshift evolution, in strong tension with our observations. The presence and redshift-dependence of correlations in the Lyman- $\alpha$ forest on $>200$ Mpc scales at $z=6$ indicates that cosmological simulations should be larger than this scale to adequately sample the Lyman- $\alpha$ forest. Despite implementing a fluctuating UVB and numerous neutral islands at $z<6$ , and matching well the sightline scatter in the Lyman- $\alpha$ forest, our fiducial SCRIPT-based simulation fails to reproduce the large-scale correlations. It may be that those ingredients are necessary, but not sufficient, to understanding the unfolding of the EoR.

</div>

<div id="div_fig1">

<img src="tmp_2507.12170/./figures/fig2.png" alt="Fig9" width="100%"/>

**Figure 9. -** Correlations in the Lyman-$\alpha$ forest flux across redshift. Top row: correlation matrix for observed QSO sightlines, showing a positive correlation (in red) in the off-diagonal terms near the diagonal at all redshifts, and an increasing correlation length at higher redshifts. Bottom row: mock sightlines with continuum-reconstruction uncertainties applied, leading to a slight increase in correlation at lower redshifts, while failing to account for the observed correlation at higher redshifts. First column: correlation matrices. Second column: standard deviation of the correlation matrices. Third column: statistical significance of the correlation, estimated as the ratio between the correlation matrix and its standard deviation. The results suggest that while continuum-reconstruction uncertainties contribute to correlations at low redshifts, they cannot account for the correlations observed at higher redshifts. (*fig:fig2*)

</div>
<div id="div_fig2">

<img src="tmp_2507.12170/./figures/fig5.png" alt="Fig1" width="100%"/>

**Figure 1. -** Amplitude (top panel) and correlation length (bottom panel) as a function of redshift for different models and simulations. The continuous gray (blue) lines are the results of fitting the observations with $\mathcal{M}2$($\mathcal{M}3$), a Gaussian model with constant (redshift-dependent) width and amplitude. The gray shaded regions represent the $1-\sigma$ error on the $A$($L$) parameters. The blue shaded region instead indicates the 16th and 84th percentiles of the  $A_0$-$A_1$($L_0$-$L_1$) joint-posterior distribution, except for redshifts $z < 5.1$, where only the $1\sigma$ upper limit of $L$ is shown. The brown dashed, the red dash-dotted and the orange dotted lines are the results of fitting the simulations ($\mathrm{const}\lambda$, $\mathrm{evolv}\lambda$, $\mathrm{SCRIPT}$) with $\mathcal{M}3$.
     (*fig:fig_tot*)

</div>
<div id="div_fig3">

<img src="tmp_2507.12170/./figures/fit_S1_M3.png" alt="Fig5.1" width="33%"/><img src="tmp_2507.12170/./figures/fit_S2_M3.png" alt="Fig5.2" width="33%"/><img src="tmp_2507.12170/./figures/fit_S3_M3.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** As Figure \ref{fig:fit_models_to_data}, but here the different simulations are fitted with model $\mathcal{M}3$.  (*fig:fit_bbbb_with_M3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.12170"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aaj}{\emph{Astronomy \& Astrophysics}}$
$\newcommand{\sci}{\emph{Science}}$
$\newcommand$
$\newcommand{\namea}[1]{\textcolor{red}{#1}}$
$\newcommand{\nameb}[1]{\textcolor{brown}{(b: #1)}}$
$\newcommand{\namec}[1]{\textcolor{cyan}{(c: #1)}}$
$\newcommand{\mala}[1]{\textcolor{black}{( #1)}}$
$\newcommand{\malb}[1]{\textcolor{brown}{( #1)}}$
$\newcommand{\aaj}{\emph{Astronomy \& Astrophysics}}$
$\newcommand{\sci}{\emph{Science}}$
$\newcommand$
$\newcommand{\namea}[1]{\textcolor{red}{#1}}$
$\newcommand{\nameb}[1]{\textcolor{brown}{(b: #1)}}$
$\newcommand{\namec}[1]{\textcolor{cyan}{(c: #1)}}$
$\newcommand{\mala}[1]{\textcolor{black}{( #1)}}$
$\newcommand{\malb}[1]{\textcolor{brown}{( #1)}}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.4}$</div>



<div id="title">

# Panchromatic characterization of the Y0 brown dwarf WISEP J173835.52+273258.9 using JWST/MIRI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.12264-b31b1b.svg)](https://arxiv.org/abs/2507.12264)<mark>Appeared on: 2025-07-17</mark> - 

</div>
<div id="authors">

M. Vasist, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Cold brown dwarf atmospheres provide a good training ground for the analysis of atmospheres of temperate giant planets. WISEP J173835.52+273258.9 (WISE 1738) is an isolated cold brown dwarf and a Y0 spectral standard with a temperature between 350–400 K, lying at the boundary of the T-Y transition. Although its atmosphere has been extensively studied in the near-infrared, its bulk physical parameters and atmospheric chemistry and dynamics are not well understood. Using a Mid-Infrared Instrument (MIRI) medium-resolution spectrum (5–18 $\mu$ m), combined with near-infrared spectra (0.98–2.2 $\mu$ m) from Hubble Space Telescope's (HST) Wide Field Camera 3 (WFC3) and Gemini Observatory's Near-Infrared Spectrograph (GNIRS), we aim to accurately characterize the atmospheric chemistry and bulk physical parameters of WISE 1738. We perform a combined atmospheric retrieval on the MIRI, GNIRS and WFC3 spectra using a machine learning algorithm called Neural Posterior Estimation (NPE) assuming a cloud-free model implemented using \texttt{petitRADTRANS} . We demonstrate how this combined retrieval approach ensures robust constraints on the abundances of major atmospheric species, the pressure-temperature ( $P$ - $T$ ) profile, bulk C/O and metalliclity [ M/H ] , along with bulk physical properties such as effective temperature, radius, surface gravity, mass and luminosity. We estimate 1D and 2D marginal posterior distributions for the constrained parameters and evaluate our results using several qualitative and quantitative Bayesian diagnostics, including Local Classifier 2-Sample Test (L-C2ST), coverage and posterior predictive checks. The combined atmospheric retrieval confirms previous constraints on $H_2$ O, $CH_4$ , $NH_3$ , and for the first time provides constraints on CO, $CO_2$ and $^{15}$ $NH_3$ . It also gives better constraints on the physical parameters and the $P$ - $T$ profile, while also revealing potential biases in characterizing objects using data from limited wavelength ranges. The retrievals further suggest the presence of disequilibrium chemistry, as evidenced by the constrained abundances of CO and CO $\(_2\)$ , which are otherwise expected to be depleted and hence not visible beyond the near-infrared wavelengths under equilibrium conditions. We estimate the physical parameters of the object as follows: an effective temperature of 402 $\(^{+12}_{-9}\)$ K, surface gravity ( $\(\log g\)$ ) of 4.43 $\(^{+0.26}_{-0.34}\)$ cm s $\(^{-2}\)$ , mass of 13 $\(^{+11}_{-7}\)$  $\(M_{\text{Jup}}\)$ , radius of 1.14 $\(^{+0.03}_{-0.03}\)$  $\(R_{\text{Jup}}\)$ , and a bolometric luminosity of $\(-6.52^{+0.05}_{-0.04}\)$  $\(\log L/L_\odot\)$ . Based on these values, the evolutionary models suggest an age between 1 and 4 Gyr, which is consistent with a high rotation rate of 6 hr of the brown dwarf. We further obtain an upper bound on the $\(^{15}\)$ NH $\(_3\)$ abundance, enabling a 3 $\sigma$ lower bound calculation of the $\(^{14}\)$ N/ $\(^{15}\)$ N ratio = 275, unable to   interpret the formation pathway as core collapse. Additionally, we calculate a C/O ratio of 1.35 $\(^{+0.39}_{-0.31}\)$ and a metallicity of 0.34 $\(^{+0.12}_{-0.11}\)$ without considering any oxygen sequestration effects.

</div>

<div id="div_fig1">

<img src="tmp_2507.12264/./plots/cloudfree/consistency_HST_Gemini_MIRI_cloudfree.png" alt="Fig9" width="100%"/>

**Figure 9. -** _Top._ WFC3+GNIRS+MIRI consistency plot. The posterior predictive distribution $p(f(\theta)+ \epsilon |x_\text{obs})$ of noisy simulations spectra for the $99.7\%$, $95\%$ and $68.7\%$ quartiles (hues of blue), overlaid on top of the the WFC3+GNIRS+MIRI observation $x_\text{obs}$(black line). {_Bottom._ Residuals of the posterior predictive samples, normalized by the inflated standard deviation of the noise distribution for each spectral channel and a horizontal line at 0 for reference (in black).
    } (*fig:cloudfree_consistency*)

</div>
<div id="div_fig2">

<img src="tmp_2507.12264/./plots/cloudfree/consistency_cloudfree_MIRIonly.png" alt="Fig10" width="100%"/>

**Figure 10. -** _Top._ MIRI consistency plot. Posterior predictive distribution $p(f(\theta)+ \epsilon |x_\text{MIRI})$ of noisy simulations spectra for the $99.7\%$, $95\%$ and $68.7\%$ quartiles (hues of blue) obtained from the retrieval on MIRI data alone extended to near-infrared wavelengths, and overlaid on top of the the WFC3+GNIRS+MIRI observation $x_\text{obs}$(black line). {_Bottom._ Residuals of the posterior predictive samples, normalized by the inflated standard deviation of the noise distribution for each spectral channel and a horizontal line at 0 for reference (in black).
    } (*fig:consistency_onlyMIRI*)

</div>
<div id="div_fig3">

<img src="tmp_2507.12264/./plots/cloudfree/consistency_cloudfree_Geminionly.png" alt="Fig11" width="100%"/>

**Figure 11. -** _Top._ GNIRS consistency plot. Posterior predictive distribution $p(f(\theta)+ \epsilon |x_\text{GNIRS})$ of noisy simulations spectra for the $99.7\%$, $95\%$ and $68.7\%$ quartiles (hues of green) obtained from the retrieval on GNIRS data alone extended to mid-infrared wavelengths, and overlaid on top of the the WFC3+GNIRS+MIRI observation $x_\text{obs}$(black line). {_Bottom._ Residuals of the posterior predictive samples, normalized by the inflated standard deviation of the noise distribution for each spectral channel and a horizontal line at 0 for reference (in black).
    } (*fig:consistency_onlyGemini*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.12264"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

144  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

19  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

8  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
